<a href="https://colab.research.google.com/github/Mukwaya-Arthur/AI_Modeling_MUK/blob/main/Autolabelling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


# Auto labelling with YOLOv8 on a OFF Road Dataset

---
## Steps

- Install YOLOv8
- Download pre-trained weights
- Import data
- Inference with Pre-trained COCO Model
- Custom Training
- Validate Custom Model
- Inference with Custom Model


Entering my directory

In [ ]:
import os
HOME = os.getcwd()
print(HOME)


/content


Mounting the drive

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


Cloning the ultralytics Github Repository for using yolov8

In [3]:
# Install the ultralytics package
!pip install ultralytics

In [4]:
!pip install -q roboflow

Creating the .yaml file for aiding in the paths
The yaml file describes our dataset, including paths to training and validation

The yaml file describes our dataset, including paths to training and validation
data, class labels


# **Custom training**

In [ ]:
import yaml
import os
from ultralytics import YOLO

# Paths for the image and mask directories
base_path = "/content/drive/MyDrive/offroad"
subsets = ["train", "valid", "test"]
paths = {subset: os.path.join(base_path, subset, "images") for subset in subsets}
mask_paths = {subset: os.path.join(base_path, subset, "masks") for subset in subsets}

# List of class names
class_names = ["bush", "grass", "sky", "terrain", "trees"]

# Number of classes
num_classes = len(class_names)

# Create data.yaml content (updated for segmentation)
data_yaml = {
    'train': paths['train'],
    'val': paths['valid'],
    'test': paths.get('test', ''),  # Optional test path
    'nc': num_classes,
    'names': {i: name for i, name in enumerate(class_names)},
    'task': 'segment',
    'train_masks': mask_paths['train'],
    'val_masks': mask_paths['valid'],
    'test_masks': mask_paths.get('test', '')  # Optional test masks path
}

# Save data.yaml
with open('coco8-seg.yaml', 'w') as file:
    yaml.dump(data_yaml, file, default_flow_style=False)

print("data.yaml created successfully.")


data.yaml created successfully.


In [ ]:
from ultralytics import YOLO

# Load a model
model = YOLO("yolov8n-seg.pt")  # load a pretrained model (recommended for training)

# Train the model
results = model.train(data="coco8-seg.yaml", epochs=100, imgsz=640)

# **Confusion matrix**

In [ ]:
from IPython.display import Image
from ultralytics import YOLO

Image(filename='/content/runs/detect/train/confusion_matrix.png', width=600)



# **Testing the model**

In [ ]:

from ultralytics import YOLO

# Load a model
model = YOLO("yolov8n-seg.pt")
model_path = '/content/runs/detect/train/weights/best.pt'
results = model(source='/content/datasets/test/images', conf=0.25,save=True)